VQEAC
=============================

In [1]:
import pennylane as qml
from pennylane import numpy as np
import optax
import jax

h2_dataset = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
symbols = ['H', 'H']
coordinates = np.array([0,0,-0.371,0,0,0.371])
h2 = h2_dataset[0]

In [2]:
h2.hf_state

tensor([1, 1, 0, 0], dtype=int64, requires_grad=True)

## VQE
Each VQE needs the following:
- An Ansatz
- Loss function

In [3]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.4602732973487343) [Z2]
+ (-0.46027329734873423) [Z3]
+ (0.236947529016103) [Z1]
+ (0.23694752901610303) [Z0]
+ (0.7735975819585081) [I0]
+ (0.14060026997649225) [Z0 Z2]
+ (0.14060026997649225) [Z1 Z3]
+ (0.18165504230123497) [Z0 Z3]
+ (0.18165504230123497) [Z1 Z2]
+ (0.18451830790068346) [Z0 Z1]
+ (0.19173391712932242) [Z2 Z3]
+ (-0.04105477232474271) [Y0 Y1 X2 X3]
+ (-0.04105477232474271) [X0 X1 Y2 Y3]
+ (0.04105477232474271) [Y0 X1 X2 Y3]
+ (0.04105477232474271) [X0 Y1 Y2 X3]


## Begin training
Let's set some expectation for the optimization process. Thankfully, $H_2$ is well studied and we have all we need in the `dataset` library to know the ground truth

### Ansatz

Before any run, we can assume that the Jordan Wigner representation `[1 1 0 0]` has the lowest energy. Let's calculate that

In [4]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit_exptected():
    qml.BasisState(h2.hf_state, wires = range(qubits))
    for op in h2.vqe_gates:
        qml.apply(op)
    return qml.expval(h2.hamiltonian)

In [5]:
print(f"HF state: {h2.hf_state}")
circuit_exptected()

HF state: [1 1 0 0]


array(-1.13637658)

In [6]:
h2.vqe_gates

[DoubleExcitation(0.27324054462951564, wires=[0, 1, 2, 3])]

Taking the superposition with themselves and the higher/lower energy level (excite/de-excite). Note that in `h2.vqe_gates` we already have the value for $\theta$

In [8]:
print(qml.draw(circuit_exptected)())

0: ─╭|Ψ⟩─╭G²(0.27)─┤ ╭<𝓗>
1: ─├|Ψ⟩─├G²(0.27)─┤ ├<𝓗>
2: ─├|Ψ⟩─├G²(0.27)─┤ ├<𝓗>
3: ─╰|Ψ⟩─╰G²(0.27)─┤ ╰<𝓗>


We would define the same circuit but without the $\theta$. Given 2 $H$ and 4 qubits, after a double excitation, the HF is the superposition of the states
$$\alpha\ket{1100}+\beta\ket{0011}:=\cos(\theta)\ket{1100}-\sin(\theta)\ket{0011}$$

[comment]: # ($\alpha\ket{110000}+\beta\ket{001100}+\gamma\ket{000011}$ this is H3)


In [9]:
@qml.qnode(dev)
def circuit(param):
    qml.BasisState(h2.hf_state, wires=range(qubits))
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.expval(H)

### Define the lost function
Remember that the lost function is the second ingredient. We use the first two equations in [this paper](https://www.nature.com/articles/s41524-023-00965-1)
$$\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle$$

$$C_1\left( {{{\mathbf{\theta }}}} \right) = \left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle + \beta \left| {\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {{\Psi}_0} \right.} \right\rangle } \right|^2$$

We can then define a lost function



At first sight, it might raises some eyebrow for someone who is from a ML background, because we define the loss function based on the predicted and the groundtruth. However we do not have any groundtruth value here. In this context, a loss function is just a function that we want to minimize.


Now we proceed to optimize the variational parameters

In [ ]:
def loss_fn_1(theta):
    return circuit(theta)

def loss_fn_2(theta, beta):
    return circuit(theta) + beta*

In [ ]:
def auxiliary_circuit(param):
    qml.BasisState(h2.hf_state, wires=range(qubits))
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.expval(H)

In [29]:
max_iterations = 100
opt = optax.sgd(learning_rate=0.4)

In [30]:
theta = np.array(0.)

# store the values of the cost function
energy = [circuit(theta)]

# store the values of the circuit parameter
angle = [theta]

opt_state = opt.init(theta)

for n in range(max_iterations):
    gradient = jax.grad(circuit)(theta)
    updates, opt_state = opt.update(gradient, opt_state)
    theta = optax.apply_updates(theta, updates)

    angle.append(theta)
    energy.append(circuit(theta))

In [31]:
theta

Array(0.11722694, dtype=float32)

In [32]:
circuit(theta)

Array(-0.898739, dtype=float32)

It is bigger than the minimum expected energy above

## Running the VQD optimization

According to equation 2 from [this paper](https://www.nature.com/articles/s41524-023-00965-1),